In [1]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd "/content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch"
#!pip3 install -r requirements.txt

Mounted at /content/gdrive
/content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch


In [2]:
!pip install mtcnn
!pip install torchfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 7.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5709 sha256=a300bd48fda88fe604c39eda505cc8775c351b7243a8e85e4c7e672f43fd42f5
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built torchfile


### Loading the data via the experiment dataloader
works fine now

In [4]:
%cd "/content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch"

import torch
import torch.nn as nn
import torch.optim as optim
from models.utils import preprocess_data, data_process
import numpy as np
import argparse
import torchvision
import cv2
from torchvision import datasets, models, transforms
import sys
import numpy
from models.vgg_face import get_pretrained_model, get_prediction
import matplotlib.pyplot as plt # D.Griesser: import added
from image_utils import save_image, show_image  #
import torch.nn.functional as F
#uncomment to see some images 
numpy.set_printoptions(threshold=sys.maxsize)


names = [line.rstrip('\n') for line in open('data/names.txt')]

batch_size = 10

model = get_pretrained_model()

model.eval()

# torch.manual_seed(1234)

dataloaders, _ = preprocess_data(batch_size)

dataiter = iter(dataloaders)
data = dataiter.next()

images, labels = data

images = images[:,[2,1,0],:,:]
show_image(images)

values, indices = get_prediction(images, model)

# predictions = [(names[index], value) for index, value in zip(indices.tolist(), values.tolist())]
predictions = [names[index] for index in indices.tolist()]
ground_truth = [names[label] for label in labels.tolist()]   

print(f"Predictions: {predictions}")
print(f"Labels: {ground_truth}")
print(f"Num. correct predictions: {(labels == indices).sum().item()}")

/content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch
['aaron_staton', 'adam_copeland', 'amitabh_bachchan', 'andrew_rannells', 'ben_whishaw', 'chris_riggi', 'connie_nielsen', 'elaine_hendrix', 'eli_roth', 'jeannie_mai', 'jennifer_ferrin', 'john_mahoney', 'josh_lucas', 'katherine_jenkins', 'kathryn_mccormick', 'kelly_rowan', 'kiersten_warren', 'lana_wachowski', 'lauren_bowles', 'lennie_james', 'marshall_allman', 'michael_gambon', 'moran_atias', 'ralf_little', 'salli_richardson-whitfield', 'tommy_morrison', 'tracy_morgan', 'virginia_williams', 'wagner_moura', 'zac_efron']
Dataset size: 900 entries


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Loading the images by hand (no dataloader/dataset)
works fine --> images correctly classified

In [ ]:
# Model classifies images correctly

import glob
import os
from google.colab.patches import cv2_imshow

labels_dict = {
    "aaron_staton": 3,
    "adam_copeland": 17,
    "amitabh_bachchan": 101,
    "andrew_rannells": 119,
    "ben_whishaw": 202,
    "chris_riggi": 366,
    "connie_nielsen": 416,
    "elaine_hendrix": 632,
    "eli_roth": 636,
    "jeannie_mai": 1007,
    "jennifer_ferrin": 1040,
    "john_mahoney": 1164,
    "josh_lucas": 1224,
    "katherine_jenkins": 1296,
    "kathryn_mccormick": 1309,
    "kelly_rowan": 1363,
    "kiersten_warren": 1389,
    "lana_wachowski": 1427,
    "lauren_bowles": 1440,
    "lennie_james": 1464,
    "marshall_allman": 1630,
    "michael_gambon": 1737,
    "moran_atias": 1807,
    "ralf_little": 2019,
    "salli_richardson-whitfield": 2163,
    "tommy_morrison": 2485,
    "tracy_morgan": 2494,
    "virginia_williams": 2543,
    "wagner_moura": 2546,
    "zac_efron": 2603
}

model = get_pretrained_model()

model.eval()

dir_list = glob.glob("/content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch/data/experiment/images/*")

for dir in dir_list:
  class_name = dir.split("/")[-1]
  files = os.listdir(dir + "/")

  for i in range(30):
    path = dir + "/" + files[i] 
    
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))

    # Forward test image through VGGFace
    image = torch.Tensor(img).permute(2, 0, 1).view(1, 3, 224, 224)
    image -= torch.Tensor(np.array([129.1863, 104.7624, 93.5940])).view(1, 3, 1, 1)

    pred = F.softmax(model(image), dim=1)
    score, index = pred.max(-1)
    print(f"Label: {class_name}; Prediction correct: {(index == labels_dict[class_name]).item()}")
    
    # cv2_imshow(img)
    # print(f"Prediction: {index}")
    # print(f"Label: {class_name}: {labels_dict[class_name]}")
  
 

### Loading the images of Wu et al. via the provided dataloader
images get classified correctly --> labels must be mapped

In [ ]:
# test with only 10 classes --> works

%cd "/content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch"

import torch
import torch.nn as nn
import torch.optim as optim
from models.utils import preprocess_data, data_process
import numpy as np
import argparse
import torchvision
import cv2
from torchvision import datasets, models, transforms
import sys
import numpy
from models.vgg_face import get_pretrained_model, get_prediction
import matplotlib.pyplot as plt # D.Griesser: import added
from image_utils import save_image, show_image  #
import torch.nn.functional as F
#uncomment to see some images 
numpy.set_printoptions(threshold=sys.maxsize)

names = [line.rstrip('\n') for line in open('data/names.txt')]

labels_dict = {
    0 : 0,
    1 : 1,
    2 : 2,
    3 : 3,
    4 : 4,
    5 : 5,
    6 : 6,
    7 : 7,
    8 : 9,
    9 : 11
}


model = get_pretrained_model()

model.eval()
batch_size = 100

dataloaders, dataset_sizes = data_process(batch_size)
data = next(iter(dataloaders["test"]))

images, labels = data
      
images = images[:,[2,1,0],:,:] #rgb to bgr

labels_mapped = torch.tensor([labels_dict[label] for label in labels.tolist()])

l_names = [names[label] for label in labels.tolist()]
m_names = [names[label] for label in labels_mapped.tolist()]

with torch.no_grad():
    model.eval()

    # outputs = model(images)
    #_, predicted = torch.max(outputs.data, 1)
    preds = F.softmax(model(images), dim=1)
    _, predicted = preds.max(-1)

print(predicted)
print(labels_mapped)
print(f"Num. correct predictions: {(labels_mapped == predicted).sum().item()}")

/content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch
['a_j__buckley', 'a_r__rahman', 'aamir_khan', 'aaron_staton', 'aaron_tveit', 'aaron_yoo', 'abbie_cornish', 'abel_ferrara', 'abigail_breslin', 'abigail_spencer']
{'test': 470}
tensor([  7,   5,   7,   5,   4,  11,   5,   3,   1,   1,   6,   0,   4,   9,
          0,   2,   4,   2,   6,   9,   9,   4,   2,   4,   7, 368,   2,   6,
          3,   7,   1,   1,   4,   5,   0,   7,   3,   4,  11,   1,   0,   0,
          2,   0,   1,   4,  11,   3,   4,   5,   2,   4,   9,   0,   1,   9,
          1,   2,  11,   4,   6,   6,   2,   1,   5,  11,   2,   6,   2,   6,
         11,   0,   3,   6,   5,  11,   3,   4,   2,   9,  11,   1,   7,   9,
         11,   1,   0,   2,   0,   6,   5,   1,   9,   4,   2,   4,   4,   6,
          6,   6])
tensor([ 7,  5,  7,  5,  4, 11,  5,  3,  1,  1,  6,  0,  4,  9,  0,  2,  4,  2,
         6,  9,  9,  4,  2,  4,  7,  4,  2,  6,  3,  7,  1,  1,  4,  5,  0,  7,
         3,  4, 11,  1,  0,  0

### Loading the experiment images via the Wu et al. dataloader (modified)
images get classified correctly --> labels must be mapped

In [ ]:
%cd "/content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch"

import torch
import torch.nn as nn
import torch.optim as optim
from models.utils import preprocess_data, data_process
import numpy as np
import argparse
import torchvision
import cv2
from torchvision import datasets, models, transforms
import sys
import numpy
from models.vgg_face import get_pretrained_model, get_prediction
import matplotlib.pyplot as plt # D.Griesser: import added
from image_utils import save_image, show_image  #
import torch.nn.functional as F
#uncomment to see some images 
numpy.set_printoptions(threshold=sys.maxsize)



names = [line.rstrip('\n') for line in open('data/names.txt')]

labels_dict = {
    0 : 3,
    1 : 17,
    2 : 101,
    3 : 119,
    4 : 202,
    5 : 366,
    6 : 416,
    7 : 632,
    8 : 636,
    9 : 1007,
    10 : 1040,
    11 : 1164,
    12 : 1224,
    13 : 1296,
    14 : 1309,
    15 : 1363,
    16 : 1389,
    17 : 1427,
    18 : 1440,
    19 : 1464,
    20 : 1630,
    21 : 1737,
    22 : 1807,
    23 : 2019,
    24 : 2163,
    25 : 2485,
    26 : 2494,
    27 : 2543,
    28 : 2546,
    29 : 2603
}


model = get_pretrained_model()

model.eval()
batch_size = 200

dataloaders, dataset_sizes = data_process(batch_size)

for i in range(5): 
  data = next(iter(dataloaders["experiment/images"]))

  images, labels = data
        
  images = images[:,[2,1,0],:,:] #rgb to bgr

  labels_mapped = torch.tensor([labels_dict[label] for label in labels.tolist()])

  label_names = [names[label] for label in labels.tolist()]
  # print(label_names)

  mapped_names = [names[label] for label in labels_mapped.tolist()]
  # print(mapped_names) 

  with torch.no_grad():
      model.eval()

      # outputs = model(images)
      #_, predicted = torch.max(outputs.data, 1)
      preds = F.softmax(model(images), dim=1)
      _, predicted = preds.max(-1)

  # print(predicted)
  # print(labels_mapped)
  print(f"Num. correct predictions: {(labels_mapped == predicted).sum().item()}")


/content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch
['aaron_staton', 'adam_copeland', 'amitabh_bachchan', 'andrew_rannells', 'ben_whishaw', 'chris_riggi', 'connie_nielsen', 'elaine_hendrix', 'eli_roth', 'jeannie_mai', 'jennifer_ferrin', 'john_mahoney', 'josh_lucas', 'katherine_jenkins', 'kathryn_mccormick', 'kelly_rowan', 'kiersten_warren', 'lana_wachowski', 'lauren_bowles', 'lennie_james', 'marshall_allman', 'michael_gambon', 'moran_atias', 'ralf_little', 'salli_richardson-whitfield', 'tommy_morrison', 'tracy_morgan', 'virginia_williams', 'wagner_moura', 'zac_efron']
{'test': 470, 'experiment/images': 900}
Num. correct predictions: 200
Num. correct predictions: 200
Num. correct predictions: 200
Num. correct predictions: 200
Num. correct predictions: 199


### Logging-Test

In [ ]:
%cd /content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch

import logging 

logger = logging.getLogger("testlogger")

logging.basicConfig(
    level=logging.INFO,
    filename="logs/test.log",
    filemode="a",
    format="%(asctime)s %(name)s %(levelname)s: %(message)s"
)


/content/gdrive/MyDrive/ColabNotebooks/BA/Experiment/vgg-face-pytorch


In [ ]:
for i in range(5):
  logger.info(f"Test Nr. {i} im test-ordner")